In [10]:
import numpy as np
import scipy
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [3]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(113, 113),nn.ReLU(True),
            nn.Linear(113, 100),nn.ReLU(True), 
            nn.Linear(100, 10)
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 100),nn.ReLU(True),
            nn.Linear(100, 113),nn.ReLU(True),
            nn.Linear(113, 113),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [4]:
num_epochs = 200
batch_size = 128
learning_rate = 1e-3
model = autoencoder()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [29]:
dataset=np.load('../data/all/all_IO_noleave_N_small.npz')
TrainX=np.asmatrix(dataset['TrainX'])[0,0].astype(np.float32)
ValidX=np.asmatrix(dataset['ValidX'])[0,0].astype(np.float32)
TestX=np.asmatrix(dataset['TestX'])[0,0].astype(np.float32)

In [30]:
TrainX

<1000x226 sparse matrix of type '<class 'numpy.float32'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [31]:
n=TrainX.shape[1]
TrainX2=scipy.sparse.vstack((TrainX[:,:int(n/2)],TrainX[:,int(n/2):])).tocoo()

In [32]:
TrainX2

<2000x113 sparse matrix of type '<class 'numpy.float32'>'
	with 10000 stored elements in COOrdinate format>

In [33]:
values=TrainX2.data
indices=np.vstack((TrainX2.row,TrainX2.col))
i=torch.LongTensor(indices)
v=torch.FloatTensor(values)
TrainXae=torch.sparse.FloatTensor(i,v,torch.Size(TrainX2.shape))

In [34]:
TrainXae

torch.sparse.FloatTensor of size (2000,113) with indices:
tensor([[   0,    0,    0,  ..., 1999, 1999, 1999],
        [  20,   26,   61,  ...,   40,   74,   83]])
and values:
tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [36]:
for epoch in range(num_epochs):
    encoding, recon = model(TrainXae)
    loss = criterion(recon, TrainXae)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
#     print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 9:
        encodingTrain, reconTrain = model(TrainXae)
        lossTrain = criterion(reconTrain, TrainXae)
        encodingValid, reconValid = model(ValidXae)
        lossValid = criterion(reconValid, ValidXae)
        encodingTest, reconTest = model(TestXae)
        lossTest = criterion(reconTest, TestXae)
        print('At epoch [{}/{}], Training loss:{:.4f}, Validation loss:{:.4f}, Test loss:{:.4f}'.format(epoch + 1, num_epochs, lossTrain.data[0],lossValid.data[0],lossTest.data[0]))        

RuntimeError: Expected object of type torch.FloatTensor but found type torch.sparse.FloatTensor for argument #2 'target'

Pytorch's support on sparse input is very poor: dataloader has issue with len(sparse_data); more importantly, loss functions expect torch.FloatTensor instead of torch.sparse.FloatTensor. I don't see a way around the latter issue.